# 说明
本篇内容基于python3.11

基于教程[Asyncio in Python - Full Tutorial - YouTube](https://www.youtube.com/watch?v=Qb9s3UiMSTA)

In [ ]:
print('hello world')

# event loop
用于管理event run or wait

# coroutines


In [ ]:
import asyncio

async def main():
	print('hello world')


# 通过async def定义的函数，只是一个coroutine object，直接使用main()并不会执行它
# 需要通过asyncio.run()来运行它


In [ ]:
main()
# print(main())
# 比如这样运行main()，只会看到<coroutine object main at xxxxxx>，并不会执行它

In [ ]:
import asyncio

"""
下面模拟一个异步操作，比如从网络获取数据
"""

async def fetch_data(delay):
    print('start fetching')
    await asyncio.sleep(delay)
    print('done fetching')
    return {'data': 'some data'}

async def main():
    print('start of main coroutine')
    task = fetch_data(2)
    print('end of main coroutine')
    result = await task
    print(f'Received result: {result}')
    

# .py里可以使用 asyncio.run(main())，但是在jupyter里只能使用await (main())，有原因，但是没看明白，以后再琢磨
await (main())

In [ ]:
import asyncio

"""
下面模拟一个异步操作，比如从网络获取数据
"""

async def fetch_data(delay):
    print('start fetching')
    await asyncio.sleep(delay)
    print('done fetching')
    return {'data': 'some data'}

async def main():
    print('start of main coroutine')
    task = fetch_data(2)
    print('end of main coroutine')
    result = await task
    print(f'Received result: {result}')

"""
调换一下 print 语句的位置，你会发现main()先把fetch_data()的任务创建好，然后立刻打印'end of main coroutine'，最后才等待fetch_data()完成并获取结果。
"""

# 正确调用 asyncio.run()
await (main())

In [ ]:
"""
现在我们写一个看起来用上了异步操作的代码，但是实际上并没有发挥异步的优势，因为我们是顺序等待每一个任务完成的。
备注：pyinstrument是用来追踪每一步代码花了多长时间的一个性能分析工具，可以通过 pip install pyinstrument 来安装
"""
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()


import asyncio

async def fetch_data(delay,id):
	print(f'start fetching {id}')
	await asyncio.sleep(delay)
	print(f'done fetching {id}')
	return {'data': f'some data {id}'}

async def main():
	task1 = fetch_data(2,1)
	task2 = fetch_data(2,2)
 
	result1 = await task1
	print(f'Received result1: {result1}')
	result2 = await task2
	print(f'Received result2: {result2}')

await main()


profiler.stop()
profiler.print()

In [ ]:
"""
现在进行优化，真正发挥异步的优势，同时发起多个任务，然后等待它们全部完成，猜猜看这次运行会花多长时间
"""
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()


import asyncio

async def fetch_data(id, sleep_time):
	print(f'coroutine {id} start fetching')
	await asyncio.sleep(sleep_time)
	return {'id':id, 'data':f'some data from coroutine {id}'}

async def main():
	task1 = asyncio.create_task(fetch_data(1,2))
	task2 = asyncio.create_task(fetch_data(2,3))
	task3 = asyncio.create_task(fetch_data(3,1))

	result1 = await task1
	result2 = await task2
	result3 = await task3

	print(result1, result2, result3)

await main()


profiler.stop()
profiler.print()

In [ ]:
"""
现在进行优简化，通过asyncio.gather()来同时等待多个任务完成
"""
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()


import asyncio

async def fetch_data(id, sleep_time):
	print(f'coroutine {id} start fetching')
	await asyncio.sleep(sleep_time)
	return {'id':id, 'data':f'some data from coroutine {id}'}

async def main():
	results = await asyncio.gather(
		fetch_data(1,2),
		fetch_data(2,3),
		fetch_data(3,1)
	)
	for result in results:
		print(f'Received result: {result}')

await main()


profiler.stop()
profiler.print()

In [14]:
"""
asyncio.gather()虽然方便了，但是debug很费劲，现在我们继续改进，用asyncio.TaskGroup代替asyncio.gather()
"""
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()


import asyncio

async def fetch_data(id, sleep_time):
	print(f'coroutine {id} start fetching')
	await asyncio.sleep(sleep_time)
	return {'id':id, 'data':f'some data from coroutine {id}'}

async def main():
	tasks=[]
	async with asyncio.TaskGroup() as tg:
		for i, sleep_time in enumerate([2,3,1], start=1):
			task = tg.create_task(fetch_data(i, sleep_time))
			tasks.append(task)
	results = [task.result() for task in tasks]
	for result in results:
		print(f'Received result: {result}')

await main()


profiler.stop()
profiler.print()

coroutine 1 start fetching
coroutine 2 start fetching
coroutine 3 start fetching
Received result: {'id': 1, 'data': 'some data from coroutine 1'}
Received result: {'id': 2, 'data': 'some data from coroutine 2'}
Received result: {'id': 3, 'data': 'some data from coroutine 3'}

  _     ._   __/__   _ _  _  _ _/_   Recorded: 18:38:57  Samples:  3
 /_//_/// /_\ / //_// / //_'/ //     Duration: 3.002     CPU time: 0.004
/   _/                      v5.1.1

Profile at /var/folders/9t/nk50v6yd64bc66hg12wnjkqm0000gn/T/ipykernel_80835/390384911.py:6

3.001 _UnixSelectorEventLoop._run_once  asyncio/base_events.py:1922
├─ 2.000 KqueueSelector.select  selectors.py:558
│  └─ 2.000 kqueue.control  <built-in>
└─ 1.002 Handle._run  asyncio/events.py:86
   └─ 1.002 fetch_data  /var/folders/9t/nk50v6yd64bc66hg12wnjkqm0000gn/T/ipykernel_80835/390384911.py:11
      └─ 1.002 sleep  asyncio/tasks.py:653
         └─ 1.002 [await]  asyncio/tasks.py




# futures